<a href="https://colab.research.google.com/github/AlexCervantesCabos/GitHub-Introduction-1/blob/main/P10_Enumeratius_AA2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="padding:30px; color: white; background-color: #0071CD">
<center>
<img src="img/logoub.jpeg"></img>
<center>
<h1>Algorísmica Avançada 2022</h1>
<h2>Problemes 10 - Enumeratius: Ramificació i Poda</h2>
</center>
</div>

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import numpy as np
from queue import PriorityQueue

<div class="alert alert-success">
    <h1>Problema 1: Job Sequencing with deadlines</h1>
    <br> Suposem que tenim un conjunt de tasques a realitzar, $J_1,\dots,J_n$. Cada una d'elles té assignada una penalització per no realitzar-la a temps $p_1,\dots,p_n$, un temps de durada de la tasca $t_1,\dots,t_n$ i un temps de deadline: $d_1,\dots,d_n$.<br><br>
    Implementeu un algoritme que trobi quines tasques hem de realitzar i quines no per a tenir una penalització mínima. La funció ha de retornar les tasques realitzades i la penalització mínima obtinguda (la suma de les tasques que descartem).
    <br>
</div>

Un exemple concret:

| Tasca (Ji)  | 1 | 2 | 3 | 4 | 
| --- | --- | --- | --- | --- |
| Penalització  | 5 | 10 | 6 | 3 | 
| Deadline  | 1 | 3 | 2 | 1 | 
| Temps  | 1 | 2 | 1 | 1 | 

Considerem la notació següent:

Sigui $x$ un node, $S_x$ les tasques agafades en el node $x$:

$$m(x) = \max\{i\ |\ i\in S_x\} \ \ \ \ \ \ \text{És l'última tasca que forma part de la solució}$$
<br>
$$s(x) = \sum_{i\in S_x}t_i \ \ \ \ \ \ \text{És la suma dels temps que tardem a fer les tasques selecionades}$$
<br>
$$d(x) = \max\{d_i\ |\ i\in S_x\} \ \ \ \ \ \ \text{És el deadline màxim de les tasques que formen part de la solució}$$
<br>
$$c(x) = \sum_{i<m,\ i\notin S_x}p_i\ \ \ \ \ \ \text{És el cost de no considerar un seguit de tasques}$$
<br>
$$u(x) = \sum_{i\notin S_x}p_i \ \ \ \ \ \ \text{És la cota superior de les penalitzacions}$$
<br>
$$sup = \infty \ \ \ \ \ \ \text{Aquí guardarem la millor cota trobada fins al moment} $$

Anem a resoldre el problema. A l'acabar, ens ha de quedar un arbre com el següent:

<center><img src="img/tree.png" width='50%'/></center>

Un parell de coses a considerar durant la implementació:
+ Us recomanem guardar 6 variables a la PriorityQueue: $c(x)$, $u(x)$, $S_x$, $s(x)$, $d(x)$ i $m(x)$. Extraureu sempre el node amb cost més baix.
+ A part de considerar les cotes $c(x)$ i $sup$ a l'hora d'afegir un nou node a la PQ, heu de controlar també els temps/deadlines. Una forma fàcil és comprovar que la suma de totes les tasques de la solució $s(x)$ és menor o igual al valor $d(x)$.

In [10]:
from queue import PriorityQueue
def compute_c(jobs, m, Sx):
    """
    Donada la llista de tasques, el valor m i les tasques seleccionades, calcula el valor c(x)
    """
    c = 0
    for i in range(m):
        if i+1 not in Sx:
            c+= jobs[i][1]
    return c

def compute_u(jobs, Sx):
    """
    Donada la llista de tasques i les tasques seleccionades, calcula el valor u(x)
    """
    u = 0
    for i in range(len(jobs)):
        if i+1 not in Sx:
            u += jobs[i][1]
    return u

def solve_jobs(jobs):
    """
    Soluciona el problema de les tasques
    """
    
    # Millor solució trobada. Inicialment té cota infinit
    sup = np.inf
    best_Sx = None
    
    # Guardem en una cua de prioritat els nodes
    # Guardarem les variables:
    # 1. Cost, c(x)
    # 2. Cota superior, u(x)
    # 3. Tasques que hem agafat fins al moment, S_x 
    # 4. Suma dels temps de les tasques de S_x, s(x)
    # 5. Màxim dels deadlines de les tasques de S_x, d(x)
    # 6. Valor m(x)
    
    # Inicialitzem la cua de prioritat i els diferents valors que guardarem
    pq = PriorityQueue()
    Sx = set()
    m = 0
    s = 0
    mx = 0
    c = compute_c(jobs, m, Sx)
    u = compute_u(jobs, Sx)
    
    pq.put((c, u, Sx, s, mx, m))
    print("c(x) u(x) S_x s(x) d(x) m(x) sup")
    while not pq.empty():
        
        # Obtenim un nou element de la cua, el de més baix cost
        current_c, current_u, current_Sx, current_sum, current_mx, current_m = pq.get()
        # EL TEU CODI AQUÍ

        if current_u < sup:
            sup = current_u
            best_Sx = current_Sx
        
        print(current_c, current_u, current_Sx, current_sum, current_mx, current_m, sup)
        
        for i in range(current_m+1, len(jobs)+1):
            new_current_Sx = current_Sx.copy()
            new_current_Sx.add(i)
            
            current_c = compute_c(jobs, current_m, new_current_Sx)
            current_u = compute_u(jobs, new_current_Sx)

            current_mx = max(jobs[i-1][2] for i in new_current_Sx)
            current_m = max(jobs[i-1][0] for i in new_current_Sx)
            
           
            # Si aquest no té un c(x) menor al sup passa a ser un node mort
            if current_c < sup and (current_sum + jobs[i-1][-1]) <= current_mx:

                pq.put((current_c, current_u, new_current_Sx, current_sum + jobs[i-1][-1], current_mx, current_m))
            
            
    return best_Sx, sup

In [11]:
# Retorna ({1, 2}, 0)
jobs = [(1, 10, 3, 2),
        (2, 3, 1, 1)]
solve_jobs(jobs)

c(x) u(x) S_x s(x) d(x) m(x) sup
0 13 set() 0 0 0 13
0 3 {1} 2 3 1 3
0 0 {1, 2} 3 3 2 0
10 10 {2} 1 1 2 0


({1, 2}, 0)

In [12]:
# Retorna ({2, 3}, 8)
jobs = [(1, 5, 1, 1),
        (2, 10, 3, 2),
        (3, 6, 2, 1),
        (4, 3, 1, 1)]
solve_jobs(jobs)

c(x) u(x) S_x s(x) d(x) m(x) sup
0 24 set() 0 0 0 24
0 19 {1} 1 1 1 19
0 9 {1, 2} 3 3 2 9
5 14 {2} 2 3 2 9
5 8 {2, 3} 3 3 3 8
10 13 {1, 3} 2 2 3 8
15 18 {3} 1 2 3 8
21 21 {4} 1 1 4 8


({2, 3}, 8)

Acabeu d'omplir la següent taula d'execució:

| Node  | S_x | m(x) | s(x) | d(x) | c(x) | u(x) | sup |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 1 | {} | - | 0 | 0 | 0 | 5+10+6+3=24 | 24 |
| 2 | {1} | 1 | 1 | 1 | 0 | 10+6+3=19 | 19 |
| 3 | ¨{2} | 2 | 2 | 3| 5 |5+6+3 = 14 | 14 |
| 4 | {3} | 3 | 1 |2 | 15 |no cal calcular, 15 > sup |14 |
| 5 | {4} | 4 | 1 | 1|  21 |21 > sup |14 |
| 6 | {1,2} | 2 | 3 | 3| 0 | 9| 9 |
| 7 | {1,3} | 3 | 2 |2 | 10 | 10 > 9 | 9|
| 8 | {1,4} | 4| 2| --|---|cas IMPOSSIBLE: s > d | 9|
| 9 | {2,3} | 3| 3 | 3| 5 |8 |8 |
| 10 | {2,4} | si seguim tots els nodes esran morts  |  | |  | | |

<div class="alert alert-success">
    <h1>Problema 2: Sliding-Puzzle</h1>
    <br>
    Donat un taulell de $n\times n$ de nombres des d'$1$ fins a $n^2-1$ inicialment desordenats, volem trobar el nombre mínim de moviments possibles de manera que els nombres estiguin ordenats en ordre creixent i la casella sense número estigui a l'última posició.
</div>

<img src="https://www.researchgate.net/profile/Ruo-Ando/publication/347300656/figure/fig1/AS:969204928901121@1608087870493/Initial-state-and-goal-state-of-8-puzzle.ppm" width='50%'/>

In [ ]:
from npuzzle import NPuzzle

# Inicialitzem un tauler i el barregem
board = NPuzzle()
board.create_board(n=3, moves=100)

print ("Un tauler aleatori:")
print(board)

Un tauler aleatori:
+-----------+
| 2 | 4 | 6 | 
+-----------+
| 7 |   | 1 | 
+-----------+
| 5 | 3 | 8 | 
+-----------+



In [ ]:
board.get_state_id()

'2,4,6,7,0,1,5,3,8'

Per a cada tauler, podem definir una cota que depèn del nombre de moviments que hem fet fins al moment i un valor optimista calculat com una 'distància' entre el tauler que estem considerant i el tauler objectiu.

$$C(X) = g(X) + h(X)$$
+ $g(X)$ és el nombre de passos que portem fins el moment.
+ $h(X)$ pot ser:
    + $h_1(X)$: El nombre de caselles que no estan al seu lloc sense tenir en compte la casella buida (hamming_distance)
    + $h_2(X)$: La suma de les distàncies de manhattan de cada casella al seu lloc correcte (manhattan_distance)

In [ ]:
board.hamming_distance() # h1(X)

8

In [ ]:
board.manhattan_distance() # h2(X)

14

Podem demanar quins moviments són valids des d'una configuració del tauler amb la funció ``allowed_moves()``. Un moviment consisteix en 'moure' la casella buida en una de les quatre direccions permeses:<br>
+ $L$: Left
+ $R$: Right
+ $U$: Up
+ $D$: Down

In [ ]:
am = board.allowed_moves()
print(am)

['L', 'R', 'U', 'D']


Executem un moviment amb la funció ``move()``

In [ ]:
new_board = board.move(am[0])
print(new_board)

+-----------+
| 2 | 4 | 6 | 
+-----------+
|   | 7 | 1 | 
+-----------+
| 5 | 3 | 8 | 
+-----------+



La funció state ens comprova si el nostre estat és un estat solució.

In [ ]:
# Solucionat: True
# No solucionat: False
new_board.state()

False

In [ ]:
import numpy as np

def solve_puzzle(board):
    """
    Soluciona el problema del N-Puzzle
    
    Params
    ======
    :board: Un objecte de la classe NPuzzle
    
    Returns
    =======
    :best_bound: Nombre de passos mínims per transformar el tauler d'entrada en el tauler objectiu
    :best_board: El tauler objectiu. Haurien de ser els números ordenats de petit a gran amb la casella buida al final.
    :expanded: El nombre de taulers expandits. Cada cop que traiem un tauler de la cua de prioritat, sumem 1.
    """
    
    # Millor solució trobada. Inicialment té cota infinit
    best_bound = np.inf
    best_board = board
    
    # Guardem en una cua de prioritat els taulells.
    # Guardarem les variables:
    # 1. Distància mínima (cota inferior) entre el tauler actual i el tauler solució
    # 2. Número de passes que duem en aquest tauler, g(X)
    # 3. El tauler
    pq = PriorityQueue()
    pq.put((board.manhattan_distance(), 0, board))
    
    # Com que els estats poden repetir-se al llarg de l'exploració, guardarem en un 'set' tots els
    # estats visitats. Així evidem tornar a visitar estats.
    existent_states = set([board.get_state_id()])
    expanded = 0
    
    while not pq.empty():
        
        # Obtenim un nou element de la cua
        curr_bound, curr_steps, curr_board = pq.get()
        expanded += 1

        # Mirem tots els moviments valids que podem fer des d'aquest tauler
        for a_move in curr_board.allowed_moves():
            new_board = curr_board.move(a_move)
            new_steps = curr_steps + 1
            new_bound = new_steps + new_board.manhattan_distance() # g(X) + h(X)
        
            # Si és un estat solució i ens millora la cota, actualitzem.
            if new_board.state():
                if new_bound < best_bound:
                    best_bound = new_bound
                    best_board = new_board
            
            # En cas de que no sigui solució però ens millori la cota. 
            elif (new_bound < best_bound) and (new_board.get_state_id() not in existent_states):
                existent_states.add(new_board.get_state_id())
                pq.put((new_bound,new_steps,new_board))
                
    return best_bound, best_board, expanded
    
    

In [ ]:
board = NPuzzle()
board.create_board(moves=100, n=3)
print("Tauler inicial:")
print(board)
distance, final_board, expanded = solve_puzzle(board)
print(f"Solucionat en {distance} passos")
print(f"Taulers expandits: {expanded}")
print("Tauler final:")
print(final_board)

Tauler inicial:
+-----------+
| 2 | 3 | 5 | 
+-----------+
| 8 | 6 | 4 | 
+-----------+
| 7 | 1 |   | 
+-----------+

Solucionat en 20 passos
Taulers expandits: 670
Tauler final:
+-----------+
| 1 | 2 | 3 | 
+-----------+
| 4 | 5 | 6 | 
+-----------+
| 7 | 8 |   | 
+-----------+



<div class="alert alert-success">    
    <h1>Problema 3: Assignació de tasques</h1>
    <br>
    Sigui $A$ una matriu de nombres enters de mida $n\times n$:
    $$A = \begin{pmatrix}a_{0,0}\quad \cdots \quad a_{0,n}\\
                         \vdots\quad\quad\quad\quad\vdots\\
                         a_{n,0}\quad \cdots \quad a_{n,n}
         \end{pmatrix}$$
    L'element $a_{i,j}$ correspon al cost d'assignar la tasca $i$ a l'empresa $j$.<br>
    Volem trobar el mínim cost d'assignar tasques a empreses amb la condició que totes les tasques han d'estar assignades i les empreses han de fer només una tasca.
     
</div>

## Funcions útils

In [ ]:
costs = np.array([[11,12,18,40],
                  [14,15,13,22],
                  [11,17,19,23],
                  [17,14,20,28]])

####  <u>np.min</u>
Ens permet obtenir el mínim de cada columna o fila d'una matriu

In [ ]:
print(costs.min(axis=0)) # Axis=0 indica que volem el mínim de cada columna
print(sum(costs.min(axis=0)))

[11 12 13 22]
58


In [ ]:
print(costs.min(axis=1)) # Axis=1 indica que volem el mínim de cada fila
print(sum(costs.min(axis=1)))

[11 13 11 14]
49


#### <u>np.delete</u>
Ens permet eliminar una fila o una columna d'un array de numpy.<br>
Observa els exemples:

In [ ]:
# Axis=0 indica que volem eliminar files. En aquest cas estem eliminant només la fila 2
new_costs1 = np.delete(costs, 2, axis=0) 
print(new_costs1)

[[11 12 18 40]
 [14 15 13 22]
 [17 14 20 28]]


In [ ]:
# Axis=0 indica que volem eliminar files. En aquest cas estem eliminant les files 0 i 1
new_costs2 = np.delete(costs, [0,1], axis=0) 
print(new_costs2)

[[11 17 19 23]
 [17 14 20 28]]


In [ ]:
# Axis=1 indica que volem eliminar columnes. En aquest cas estem eliminant les columnes 0 i 2
new_costs3 = np.delete(costs, [0,2], axis=1)
print(new_costs3)

[[12 40]
 [15 22]
 [17 23]
 [14 28]]


In [ ]:
# Podem esborrar files i columnes d'una matriu
new_costs4 = np.delete(np.delete(costs, [0,1], axis=0), [2,3], axis=1)
print(new_costs4)

[[11 17]
 [17 14]]


In [ ]:
def inf_bound(matrix):
    """
    Calcula la suma del mínim de cada columna
    
    Params
    ======
    :matrix: La matriu de costs
    
    Returns
    =======
    :inf: La suma del mínim de cada columna
    """
    if len(matrix)==0:
        return 0
    # retorna els minims de cada columna print(np.min(matrix, axis = 0))
    return sum(np.min(matrix, axis=0))

def sup_bound(matrix):
    """
    Retorna el cost d'una assignació qualsevol.
    
    Params
    ======
    :matrix: La matriu de costs
    
    Returns
    =======
    :sup: El cost d'una assignació qualsevol. Per exemple, podem retornar la suma de la diagonal de la matriu
          que consisteix en assignar la tasca 'i' a l'empresa 'i' on i=0,1,2,3,4...
    """
    #Trace és una funció que retorna la traça de qualsevol matriu quadrada
    return matrix.trace()

def tasks(matrix):
    """
    Troba l'assignació entre tasques i empreses amb cost mínim utilitzant ramificació i poda.
    Cada cop que troba una assignació millor ll'imprimeix per pantalla.
    
    Params
    =====
    :matrix: La matriu de costs
    """
    
    # Cotes inicials
    sup = sup_bound(matrix)
    inf = inf_bound(matrix)

    # Cua de prioritat. Guardarem quatre elements:
    # 1. Prioritat
    # 2. Parelles ja assignades (tasca, empresa)
    # 3. Tasca que hem d'assignar a continuació (row)
    # 4. Empreses ja assignades (col)
    pq = PriorityQueue()
    pq.put((inf, [], 0, set([])))
    
    # Iterarem mentre la cua de prioritat no sigui buida
    while not pq.empty():
        
        # Extraiem un element
        elem_cota, elem_list, elem_row, elem_cols = pq.get()  

        for i in range(matrix.shape[0]):

          if i not in elem_cols:
          
            new_elem_list = elem_list.copy()
            new_elem_list.append((elem_row, i))

            new_elem_row = elem_row + 1

            new_elem_cols = elem_cols.copy()
            new_elem_cols.add(i)

            reduced_matrix = np.delete(np.delete(matrix, list(range(0,new_elem_row)), axis=0), list(new_elem_cols), axis=1)
            new_elem_cota = sum(matrix[i,j] for i,j in new_elem_list) + inf_bound(reduced_matrix)
            

            if new_elem_cota < sup:

              if len(new_elem_list) == matrix.shape[0]:
                sup = new_elem_cota
                solucio = new_elem_list
              
              else:
                pq.put((new_elem_cota, new_elem_list, new_elem_row, new_elem_cols))
    
    return solucio
        

In [ ]:
costs = np.array([[11,12,18,40],
                  [14,15,13,22],
                  [11,17,19,23],
                  [17,14,20,28]])

print("Matriu de costs:")
print(costs)
print()

# Millor solució trobada, cost: 61
# Tasca 0 -> Empresa 0
# Tasca 1 -> Empresa 2
# Tasca 2 -> Empresa 3
# Tasca 3 -> Empresa 1

tasks(costs)

Matriu de costs:
[[11 12 18 40]
 [14 15 13 22]
 [11 17 19 23]
 [17 14 20 28]]



[(0, 0), (1, 2), (2, 3), (3, 1)]

In [ ]:
costs = np.array( [[38, 43, 49, 21, 25, 26, 18, 49],
                   [32, 48, 34, 38, 29, 16, 45, 44],
                   [45, 16, 29, 25, 39, 29, 32, 34],
                   [44, 30, 41, 36, 27, 34, 33, 24],
                   [34, 43, 39, 10, 23, 17, 39, 23],
                   [26, 28, 36, 45, 27, 47, 36, 45],
                   [28, 22, 42, 10, 38, 19, 38, 25],
                   [19, 36, 21, 46, 13, 39, 30, 24]])

print("Matriu de costs:")
print(costs)
print()

# Millor solució trobada, cost: 154
# Tasca 0 -> Empresa 6
# Tasca 1 -> Empresa 5
# Tasca 2 -> Empresa 1
# Tasca 3 -> Empresa 7
# Tasca 4 -> Empresa 4
# Tasca 5 -> Empresa 0
# Tasca 6 -> Empresa 3
# Tasca 7 -> Empresa 2

tasks(costs)

Matriu de costs:
[[38 43 49 21 25 26 18 49]
 [32 48 34 38 29 16 45 44]
 [45 16 29 25 39 29 32 34]
 [44 30 41 36 27 34 33 24]
 [34 43 39 10 23 17 39 23]
 [26 28 36 45 27 47 36 45]
 [28 22 42 10 38 19 38 25]
 [19 36 21 46 13 39 30 24]]



[(0, 6), (1, 5), (2, 1), (3, 7), (4, 4), (5, 0), (6, 3), (7, 2)]

En aquest cas no us donem la solució ja que és un exemple aleatori. Proveu-ho amb diferents mides de matrius.

In [ ]:
costs = np.random.randint(10,50,(10,10))

print("Matriu de costs:")
print(costs)
print()

tasks(costs)

Matriu de costs:
[[16 36 33 19 38 47 40 49 18 36]
 [19 48 14 45 38 24 26 49 25 35]
 [16 11 12 12 11 11 47 36 10 33]
 [48 19 35 48 24 45 35 40 13 28]
 [46 48 44 11 42 32 36 22 37 24]
 [12 47 35 22 14 49 24 47 41 19]
 [14 18 30 29 27 23 20 48 22 18]
 [31 22 20 41 30 28 40 25 24 17]
 [36 33 27 44 31 41 42 43 49 23]
 [37 12 29 31 37 43 33 41 47 48]]



[(0, 0),
 (1, 2),
 (2, 5),
 (3, 8),
 (4, 3),
 (5, 4),
 (6, 6),
 (7, 7),
 (8, 9),
 (9, 1)]